1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


Ответ: Hit rate@k — количество купленных товаров среди топ-k рекомендованных;
Precision@k — количество первых k рекомендованных товаров, которые были куплены к общему числу k рекомендованных товаров.
2-3 примера задач, в которых более уместно использовать метрику hit_rate:
1. рекомендация товаров на сайте/приложении магазина;
2. рекомендация книг в книжном магазине.

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Ответ: Для учета местоположения элемента.
Степенную.
Функция потерь.

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Ответ: Precision, recall, roc_auc, f1_score.

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

In [1]:
import pandas as pd
import numpy as np

In [2]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [8]:
print(type(boughted))

<class 'list'>


In [12]:
l = 0
for i in boughted:
    for k in recommended[:8]:
        if i == k:
            l += 1
# print(l)
pr = l / len(recommended[:8])
print(pr)

0.5


In [16]:
def precision_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [17]:
precision_at_k(recommended, boughted, k=8)

0.5

In [20]:
l = 0
for i in boughted:
    for k in recommended[:8]:
        if i == k:
            l += 1
# print(l)
rec = l / len(boughted)
print(rec)

0.6666666666666666


In [21]:
def recall_at_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [22]:
recall_at_k(recommended, boughted, k=8)

0.6666666666666666

In [35]:
l = []
l = np.isin(recommended[:8], boughted)

# print(l)

for i in range(1, 8+1): 
    print(i)
    
    if l[i-1] == True:
        s1 = 0
        s2 = 0
        for i in boughted:
            for k in recommended[:i]:
                if i == k:
                    s1 += 1
        # print(l)
        pr = s1 / len(recommended[:i])
        print(pr)
        s2 += pr
return s2 / len(l)

1
2
0.4
3
0.4
4
5
0.4
6
0.4
7
8


SyntaxError: 'return' outside function (<ipython-input-35-c7bf42025069>, line 20)

In [23]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        print(i)
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            print(p_k)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [24]:
ap_k(recommended, boughted, k=8)

1
2
0.5
3
0.6666666666666666
4
5
0.6
6
0.6666666666666666
7
8


0.30416666666666664

In [65]:
def dcg(bought_list, k=8):
    bought_list = np.asarray(bought_list)[:k]
    n_bought_list = len(bought_list)
    if n_bought_list == 0:
        return 0
    discounts = np.log2(np.arange(n_bought_list) + 2)
    return np.sum(bought_list / discounts)

def ndcg(bought_list, k=8):
    for i in range(k):
        best_dcg = dcg(sorted(bought_list, reverse=True), i)
        if best_dcg == 0:
            print('На ноль делить нельзя')
        else:
            print(dcg(bought_list, i) / best_dcg)

In [66]:
ndcg(boughted, k=8)

На ноль делить нельзя
0.09090909090909091
0.17344558523356043
0.2672559703013606
0.37648303208157863
0.5060890538800328
0.6628286376536807
0.6628286376536807


In [41]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=8
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [42]:
reciprocal_rank(recommended, boughted)

0.9299999999999999